<a href="https://colab.research.google.com/github/mistryvivek/YRKCS-PRBX/blob/main/Model_1a_Basic_NN_(Earth_movers_distance).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import numpy as np
import random
import pandas as pd
import random
from sklearn.preprocessing import LabelEncoder

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cpu


# Setup Weights and Bias.

* AI experimental tool.

In [4]:
!pip install wandb -qU

In [5]:
# Log in to your W&B account
import wandb
import random
import math

In [6]:
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Earth Movers Distance Loss

* https://github.com/TakaraResearch/Pytorch-1D-Wasserstein-Statistical-Loss/blob/master/pytorch_stats_loss.py

In [7]:
import torch
from torch.autograd import Variable


#######################################################
#       STATISTICAL DISTANCES(LOSSES) IN PYTORCH      #
#######################################################

## Statistial Distances for 1D weight distributions
## Inspired by Scipy.Stats Statistial Distances for 1D
## Pytorch Version, supporting Autograd to make a valid Loss
## Supposing Inputs are Groups of Same-Length Weight Vectors
## Instead of (Points, Weight), full-length Weight Vectors are taken as Inputs
## Code Written by E.Bao, CASIA

def torch_wasserstein_loss(tensor_a,tensor_b):
    #Compute the first Wasserstein distance between two 1D distributions.
    return(torch_cdf_loss(tensor_a,tensor_b,p=1))

def torch_energy_loss(tensor_a,tensor_b):
    # Compute the energy distance between two 1D distributions.
    return((2**0.5)*torch_cdf_loss(tensor_a,tensor_b,p=2))

def torch_cdf_loss(tensor_a,tensor_b,p=1):
    # last-dimension is weight distribution
    # p is the norm of the distance, p=1 --> First Wasserstein Distance
    # to get a positive weight with our normalized distribution
    # we recommend combining this loss with other difference-based losses like L1

    # normalize distribution, add 1e-14 to divisor to avoid 0/0
    tensor_a = tensor_a / (torch.sum(tensor_a, dim=-1, keepdim=True) + 1e-14)
    tensor_b = tensor_b / (torch.sum(tensor_b, dim=-1, keepdim=True) + 1e-14)
    # make cdf with cumsum
    cdf_tensor_a = torch.cumsum(tensor_a,dim=-1)
    cdf_tensor_b = torch.cumsum(tensor_b,dim=-1)

    # choose different formulas for different norm situations
    if p == 1:
        cdf_distance = torch.sum(torch.abs((cdf_tensor_a-cdf_tensor_b)),dim=-1)
    elif p == 2:
        cdf_distance = torch.sqrt(torch.sum(torch.pow((cdf_tensor_a-cdf_tensor_b),2),dim=-1))
    else:
        cdf_distance = torch.pow(torch.sum(torch.pow(torch.abs(cdf_tensor_a-cdf_tensor_b),p),dim=-1),1/p)

    cdf_loss = cdf_distance.mean()
    return cdf_loss

def torch_validate_distibution(tensor_a,tensor_b):
    # Zero sized dimension is not supported by pytorch, we suppose there is no empty inputs
    # Weights should be non-negetive, and with a positive and finite sum
    # We suppose all conditions will be corrected by network training
    # We only check the match of the size here
    if tensor_a.size() != tensor_b.size():
        raise ValueError("Input weight tensors must be of the same size")

## Modify code for sliding window approach (like wk11 code).

* Look at the full race image instead.
* What data types are in the wk11 practical.
* No vocab size required because they apply to catogorical contexts.

In [8]:
BASE_PATH = r"/content/drive/MyDrive/prbx_data/v1/"
max_race_size = 0
max_tyre_life = 0

# Compound Encoder

In [9]:
compound_encoder = LabelEncoder()
all_compounds = ['SOFT', 'MEDIUM', 'HARD', 'INTERMEDIATE', 'WET']
compound_encoder.fit(all_compounds)

LabelEncoder()

In [10]:
# TRAINING
training_inputs = []
training_outputs = []

RaceCalender22 = pd.read_csv(BASE_PATH + r"2022/eventCalender2022.csv")
for _, row in RaceCalender22.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2022/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values  # Add the compound data

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1] else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            encoded_compound = compound_encoder.transform(compound_array)
            # Combine tyre_life and compound as input features
            combined_input = np.column_stack((tyre_life_array, encoded_compound))  # Combine into a 2D array
            training_inputs.append(combined_input)
            training_outputs.append(stint_change_array)

In [11]:
# TESTING
testing_inputs = []
testing_outputs = []

RaceCalender23 = pd.read_csv(BASE_PATH + r"2023/eventCalender2023.csv")
for _, row in RaceCalender23.iterrows():
    if row['EventFormat'] != 'testing':
        TempRaceLoad = pd.read_csv(
            BASE_PATH + f"2023/{row['RoundNumber']}_{row['OfficialEventName']}/{row['RoundNumber']}_{row['OfficialEventName']}_Race.csv".replace(" ", "_")
        )
        for driver in TempRaceLoad['Driver'].unique():
            TempRaceLoadDriver = TempRaceLoad[TempRaceLoad['Driver'] == driver].sort_values(by='LapNumber', ascending=True)
            tyre_life_array = TempRaceLoadDriver['TyreLife'].values
            stint_array = TempRaceLoadDriver['Stint'].values
            compound_array = TempRaceLoadDriver['Compound'].values  # Add the compound data

            if max(TempRaceLoadDriver['LapNumber']) > max_race_size:
                max_race_size = max(TempRaceLoadDriver['LapNumber'])

            if max(TempRaceLoadDriver['TyreLife']) > max_tyre_life:
                max_tyre_life = max(TempRaceLoadDriver['TyreLife'])

            stint_change_array = [0 if stint_array[i] == stint_array[i + 1] else 1
                                  for i in range(len(stint_array) - 1)] + [0]
            stint_change_array = np.array(stint_change_array)

            encoded_compound = compound_encoder.transform(compound_array)
            # Combine tyre_life and compound as input features
            combined_input = np.column_stack((tyre_life_array, encoded_compound))  # Combine into a 2D array
            testing_inputs.append(combined_input)
            testing_outputs.append(stint_change_array)

In [12]:
class RNNv1bDropout(nn.Module):
    def __init__(self,input_size,hidden_size,dropout_prob): #vocab_size
        super(RNNv1bDropout, self).__init__()
        #self.embedding =  nn.Embedding(vocab_size, hidden_size)
        self.linear_x = nn.Linear(input_size, hidden_size)
        self.linear_h = nn.Linear(hidden_size,hidden_size)
        self.linear_y = nn.Linear(hidden_size,1)
        self.tanh = nn.Tanh()
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(dropout_prob)

    def forward(self,x,hprev):
        """
        h = self.tanh(self.embedding(x) + self.linear_h(hprev))
        y = self.linear_y(h)
        """
        h = self.tanh(self.linear_x(x) + self.linear_h(hprev))
        h - self.dropout(h)
        y = self.sigmoid(self.linear_y(h))
        return h,y

In [14]:
def calculate_loss(model,inputs,targets,hprev,vocab_size):
  loss_func = torch_wasserstein_loss
  seq_length = len(inputs)
  outputs = []
  for t in range(seq_length):
    # For each character in the input sequence, pass through RNN with previous hidden state
    hprev,y = model(inputs[t],hprev)
    # Gradually build up matrix of output logits of size seq_length * vocab_size - we want it at every time step do not want this.
    # Compute cross entropy loss for seq_length actual targets against estimated distributions
    # RuntimeError: result type Float can't be cast to the desired output type Long
    outputs.append(y)

  outputs = torch.stack(outputs).squeeze(1)
  loss = loss_func(targets, outputs)

  # For truncated backprop, the next subsequence will use the final hidden state
  # but will not backprop through it so we need to detach
  hprev = hprev.detach()

  return loss, hprev

In [15]:
# Model parameters to be logged by w and b.
hidden_size = 6
lr = 0.002
iterations = 10000
input_parameters = ['TyreLife', 'Compound']
dataset = 'v1'
input_size = 2
dropout = 0.45

In [16]:
# PADDING
def add_padding(max_race_size, sequence):
    if len(sequence) >= max_race_size:
        return sequence[:max_race_size]
    else:
        # Handle both 1D and 2D arrays
        if sequence.ndim == 1:  # If 1D array
            padding_shape = (max_race_size - len(sequence),)
        else:  # If 2D array
            padding_shape = (max_race_size - len(sequence), sequence.shape[1])

        padding = np.zeros(padding_shape, dtype=sequence.dtype)
        return np.concatenate([sequence, padding])

In [17]:
def check_accuracy(model):
  total, matches = 0, 0
  no_pit_prediction_total, correct_no_pit_prediction = 0,0
  pit_prediction_total, correct_pit_prediction = 0,0

  for t in range(len(testing_inputs) - 1):
      inputs = add_padding(int(max_race_size),testing_inputs[t])
      outputs = add_padding(int(max_race_size),testing_outputs[t])
      hprev = torch.zeros(hidden_size, device=device)

      for x in range(len(inputs) - 1):

        hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.float32), hprev)

        if (y.item() > 0.5):
          pit_prediction_total += 1
          if 1 in outputs[max(0, x - 2): min(int(max_race_size), x+4)]:
              correct_pit_prediction += 1
        else:
          no_pit_prediction_total += 1
          if y.floor().item() == outputs[x]:
              correct_no_pit_prediction += 1

  if (pit_prediction_total + no_pit_prediction_total) != 0:
      overall_accuracy = (correct_no_pit_prediction + correct_pit_prediction) / (no_pit_prediction_total + pit_prediction_total) * 100
      print(f"Overall Accuracy: {overall_accuracy:.2f}%")
      wandb.log({"overall_accuracy": overall_accuracy})
  else:
      print("Overall Accuracy: 0.0%")
      wandb.log({"overall_accuracy": 0.0})

  if no_pit_prediction_total != 0:
    print(f"Total No Pit Accuracy: {correct_no_pit_prediction / no_pit_prediction_total * 100}")
    wandb.log({"total no pit accurary (exact match)": correct_no_pit_prediction / no_pit_prediction_total * 100})
  else:
    print(f"Total No Pit Accuracy: 0.0")
    wandb.log({"total no pit accurary (exact match)": 0.0})

  if pit_prediction_total != 0:
    print(f"Total Pit Accuracy: {correct_pit_prediction / pit_prediction_total * 100}")
    wandb.log({"total pit accurary (3 laps either side)": correct_pit_prediction / pit_prediction_total * 100})
  else:
    print(f"Total Pit Accuracy: 0.0")
    wandb.log({r"total pit accurary (3 laps either side)": 0.0})

  print(f"Total Pit Predictions: {pit_prediction_total}")
  wandb.log({"total pit predictions": pit_prediction_total})

In [19]:
def train(training_inputs, training_outputs):
  model = RNNv1bDropout(input_size,hidden_size,dropout)#vocab_size
  model.to(device)

  optim = torch.optim.Adagrad(model.parameters(), lr = lr)

  config = {
      "learning_rate": lr,
      "hidden_size": hidden_size,
      "iterations": 10000,
      "input_parameters": input_parameters,
      "dataset": dataset,
      "input_size": input_size,
      "loss_function": "torch_wasserstein_loss",
      "dropout": dropout
  }

  wandb.init(project="binary_classifier_emd", config=config)

  # Shuffle the data before starting the training loop
  training_data = list(zip(training_inputs, training_outputs))
  random.shuffle(training_data)
  training_inputs, training_outputs = zip(*training_data)

  n, p = 0, 0
  while n<=iterations:
    hprev = torch.zeros(hidden_size, device=device) # reset RNN memory

    training_data = list(zip(training_inputs, training_outputs))
    random.shuffle(training_data)
    training_inputs, training_outputs = zip(*training_data)

    # Why apply padding here
    inputs = add_padding(int(max_race_size),training_inputs[p])
    outputs = add_padding(int(max_race_size),training_outputs[p])

    # Extract next subsequence of characters
    inputs = torch.tensor(training_inputs[p],dtype=torch.float32, device=device)
    targets = torch.tensor(training_outputs[p],dtype=torch.float32, device=device)

    # Compute loss for current subsequence
    loss, hprev = calculate_loss(model,inputs,targets,hprev,input_size)
    optim.zero_grad()
    loss.backward()
    optim.step()

    wandb.log({"loss": loss.item()})

    if n % 500 == 0:
      print('[{:}] Loss: {:.2f}'.format(n,loss.item()))
      check_accuracy(model)

    p += 1 # move data pointer
    n += 1 # iteration counter

    if p >= len(training_inputs):  # Check if we've gone through all data
      # Shuffle the data before starting a new epoch
      training_data = list(zip(training_inputs, training_outputs))
      random.shuffle(training_data)
      training_inputs, training_outputs = zip(*training_data)

      p = 0

  # Optionally, save the entire PyTorch model object
  torch.save(model, "full_model.pth")
  wandb.save("full_model.pth")

#train(training_inputs, training_outputs)

In [20]:
wandb.finish()

In [21]:
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',
    'metric': {
        'name': 'loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-4,
            'max': 1e-2
        },
        'optimizer': {
            'values': ['adam', 'sgd', 'rmsprop', 'adagrad']
        },
        'hidden_size': {
            'min': 1,
            'max': 50
        },
        'dropout': {
            'min': 0.1,
            'max': 0.5
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="binary_classifier_emd")

Create sweep with ID: i1dywmsy
Sweep URL: https://wandb.ai/vm800-university-of-york/binary_classifier_emd/sweeps/i1dywmsy


In [22]:
# Run the sweep
wandb.agent(sweep_id, function=lambda: train(training_inputs, training_outputs), count=10)

wandb: Agent Starting Run: 54c0gw4a with config:
wandb: 	dropout: 0.3683225472176558
wandb: 	hidden_size: 9
wandb: 	learning_rate: 0.005866980758008903
wandb: 	optimizer: rmsprop
wandb: WARNING Ignoring project 'binary_classifier_emd' when running a sweep.
wandb: Currently logged in as: vm800 (vm800-university-of-york). Use `wandb login --relogin` to force relogin


[0] Loss: 18.28


<ipython-input-17-e7617c931dd8>:13: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  hprev, y = model(torch.tensor([inputs[x]], device=device, dtype=torch.float32), hprev)


Overall Accuracy: 64.47%
Total No Pit Accuracy: 98.62340719512818
Total Pit Accuracy: 17.160476325489086
Total Pit Predictions: 14108
[500] Loss: 9.81
Overall Accuracy: 97.10%
Total No Pit Accuracy: 97.28968039815314
Total Pit Accuracy: 75.2542372881356
Total Pit Predictions: 295
[1000] Loss: 7.27
Overall Accuracy: 96.13%
Total No Pit Accuracy: 97.23712857624727
Total Pit Accuracy: 61.46435452793835
Total Pit Predictions: 1038
[1500] Loss: 6.47
Overall Accuracy: 96.16%
Total No Pit Accuracy: 97.26760823409951
Total Pit Accuracy: 64.8695652173913
Total Pit Predictions: 1150
[2000] Loss: 9.68
Overall Accuracy: 96.16%
Total No Pit Accuracy: 97.26335924966062
Total Pit Accuracy: 67.17865804365401
Total Pit Predictions: 1237
[2500] Loss: 10.19
Overall Accuracy: 95.83%
Total No Pit Accuracy: 97.26623174899038
Total Pit Accuracy: 64.15352981494175
Total Pit Predictions: 1459
[3000] Loss: 16.99
Overall Accuracy: 95.82%
Total No Pit Accuracy: 97.26044455513355
Total Pit Accuracy: 65.48788474132

loss,▃▂▂▃▂▁▂▇▂▃▃▅▄▆▆▂▃▂▆▃▂▄█▄▂▅▂▃▆▃▆▆▄▁▆▆▇▂▂▅
overall_accuracy,▁███████▇▇▇▇▇▇▇▇▇▆▆▆▆
total no pit accurary (exact match),█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
total pit accurary (3 laps either side),▁█▆▇▇▇▇▆▅▄▄▄▄▅▅▄▄▄▄▄▄
total pit predictions,█▁▁▁▁▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃
loss,5.61241
overall_accuracy,89.51529
total no pit accurary (exact match),97.24459
total pit accurary (3 laps either side),39.0604
total pit predictions,4470


...

[0] Loss: 14.73
Overall Accuracy: 14.49%
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[500] Loss: 14.30
Overall Accuracy: 58.81%
Total No Pit Accuracy: 95.98795310315155
Total Pit Accuracy: 12.886084357356358
Total Pit Predictions: 15055
[1000] Loss: 14.84
Overall Accuracy: 59.98%
Total No Pit Accuracy: 96.07090258783926
Total Pit Accuracy: 13.103778096604495
Total Pit Predictions: 14637
[1500] Loss: 14.27
Overall Accuracy: 59.95%
Total No Pit Accuracy: 96.07708914749091
Total Pit Accuracy: 13.132760267430754
Total Pit Predictions: 14658
[2000] Loss: 15.22
Overall Accuracy: 59.48%
Total No Pit Accuracy: 96.07082092726499
Total Pit Accuracy: 13.098847786537297
Total Pit Predictions: 14841
[2500] Loss: 8.71
Overall Accuracy: 59.48%
Total No Pit Accuracy: 96.07572051472934
Total Pit Accuracy: 13.1105571649936
Total Pit Predictions: 14843
[3000] Loss: 7.52
Overall Accuracy: 59.47%
Total No Pit Accuracy: 96.07957870099474
Total Pit Accuracy:

loss,▃▆▁▄▄▂▂▄▆▇▅▃▃▇▅▅▁▇▇▄▃▄▆▅█▇▄▁▅▂▄▃▅▇▃▇▄▆▃▅
overall_accuracy,▁██████████████▇▇▇▇▇▇
total no pit accurary (exact match),▁████████████████████
total pit accurary (3 laps either side),█▁▂▂▂▂▂▂▁▁▁▂▂▂▂▂▂▂▂▂▂
total pit predictions,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂
loss,6.07447
overall_accuracy,55.25573
total no pit accurary (exact match),95.89833
total pit accurary (3 laps either side),13.18495
total pit predictions,16534


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: urkp4bqw with config:
wandb: 	dropout: 0.11117494162504316
wandb: 	hidden_size: 47
wandb: 	learning_rate: 0.00599604106655897
wandb: 	optimizer: rmsprop
wandb: WARNING Ignoring project 'binary_classifier_emd' when running a sweep.


[0] Loss: 6.34
Overall Accuracy: 14.49%
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[500] Loss: 13.06
Overall Accuracy: 14.49%
Total No Pit Accuracy: 0.0
Total Pit Accuracy: 14.493744242027995
Total Pit Predictions: 33649
[1000] Loss: 10.45
Overall Accuracy: 14.49%
Total No Pit Accuracy: 100.0
Total Pit Accuracy: 14.486120192593473
Total Pit Predictions: 33646
[1500] Loss: 10.75
Overall Accuracy: 14.49%
Total No Pit Accuracy: 100.0
Total Pit Accuracy: 14.486120192593473
Total Pit Predictions: 33646
[2000] Loss: 7.58
Overall Accuracy: 14.49%
Total No Pit Accuracy: 100.0
Total Pit Accuracy: 14.486120192593473
Total Pit Predictions: 33646
[2500] Loss: 8.19
Overall Accuracy: 14.49%
Total No Pit Accuracy: 100.0
Total Pit Accuracy: 14.481036737605516
Total Pit Predictions: 33644
[3000] Loss: 14.28
Overall Accuracy: 33.14%
Total No Pit Accuracy: 95.81622176591375
Total Pit Accuracy: 14.251459952817418
Total Pit Predictions: 25857
[3500] Loss:

loss,▂█▄▅▃▅▄▄▂▃▃▅▃▃▃▅▁▂▃▂▄▃▂▄▂▁▃▂▂▄▂▄▄▂▂▃▅▄▄▂
overall_accuracy,▁▁▁▁▁▁▅▆▇▇███████████
total no pit accurary (exact match),▁▁███████████████████
total pit accurary (3 laps either side),██████▆▅▂▁▃▁▁▂▂▂▂▂▂▂▂
total pit predictions,██████▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁
loss,10.18532
overall_accuracy,47.97765
total no pit accurary (exact match),95.94344
total pit accurary (3 laps either side),13.79345
total pit predictions,19647
